In [ ]:
import h5py as h5
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams.update({'font.size': 14})

import numpy as np
from rsbeams.rsstats import kinematic
from pywarpx import picmi
from scipy.fft import fft, fftfreq

import matplotlib.colors as mcolors

In [ ]:
filenames = ['<diags pathway>'] 
labels = ['<label>']
figure_folder = 'figures/<figures subfolder>'
dt = 0.5e-10

In [ ]:
diff_step = 0
current_step = 2151000

In [ ]:
current_step*dt*1e6

In [ ]:
step_min = 0
steps = np.arange(step_min,current_step+500,500)

In [ ]:
species_names = np.array(['deuterium','argon','electron'])
colors = np.array(['orange', 'r', 'b'])

In [ ]:
plt.figure(figsize=(12,6))
time = steps*dt*1e6
for index_filename,filename in enumerate(filenames):
    num_species = np.zeros((len(steps),len(species_names)))
    nmp_species = np.zeros((len(steps),len(species_names)))
    for index_step, step in enumerate(steps):
        data_step = h5.File(f'{filename}/particle/openpmd_{step:0>6}.h5', 'r')
        for index_species, species_name in enumerate(species_names): 
            weights = data_step[f'data/{step}/particles/{species_name}/weighting']
            n_particles = np.sum(weights)
            n_macroparticles = len(weights)
            num_species[index_step,index_species] = n_particles
            nmp_species[index_step,index_species] = n_macroparticles

    plt.subplot(1,2,1)
    plt.plot(time, num_species[:,0], 'orange', label='deuterium')
    plt.plot(time, num_species[:,1], 'r', label='ar ions')
    plt.plot(time, num_species[:,2], ':b', label='electrons')
    plt.legend()
    plt.xlabel('time (us)')
    plt.ylabel('n particles')
    plt.grid()
    
    plt.subplot(1,2,2)
    plt.plot(time, nmp_species[:,0], 'orange', label='deuterium')
    plt.plot(time, nmp_species[:,1], 'r', label='ar ions')
    plt.plot(time, nmp_species[:,2], ':b', label='electrons')
    plt.legend()
    plt.xlabel('time (us)')
    plt.ylabel('n macroparticles')
    plt.grid()
    
    plt.suptitle(labels[index_filename])

plt.tight_layout()
plt.savefig(f'{figure_folder}/particle_counts.png')
plt.show()

In [ ]:
fusion_multiplier=1e20
data = np.array([])
for index_step, step in enumerate(steps):
    directory_path = f'{filenames[0]}'
    
    if step == step_min:
        continue
    data_temp = np.load(f'{directory_path}/particle_count_helium3_{step}.npy')
    data = np.append(data, data_temp[1])

x = np.arange(step_min+1,current_step+1,1)*dt
y = data/fusion_multiplier
    
z = np.polyfit(x, np.cumsum(y), 3)
p = np.poly1d(z)
fit = p(x)
        
plt.figure(figsize=(10,5))
plt.plot(x*1e6,np.cumsum(y),'k', label='data')
# plt.plot(x*1e6,fit,'--r', label='cubic fit')
plt.legend()
plt.xlabel('t (us)')
plt.ylabel(f'Helium3 Physical Particles')
plt.title('WarpX fusion: on')
plt.grid()
plt.tight_layout()
plt.savefig(f'{figure_folder}/he3_timeseries.png')
plt.show()

In [ ]:
def efield_fft(filenames, figure_folder, steps, efield_component, dt, x_index, y_index, z_index, suffix):
    data = np.array([])
    directory_path = f'{filenames[0]}'
    for step in steps:
        if step == np.min(steps):
            data_step = h5.File(f'{directory_path}/field/openpmd_{steps[0]:0>6}.h5', 'r')
            E = data_step[f'data/{steps[0]}/fields/E/{efield_component}'][()].T

            x_index += int(np.shape(E)[0]/2)
            y_index += int(np.shape(E)[1]/2)
            z_index += int(np.shape(E)[2]/2)
        
        data_step = h5.File(f'{directory_path}/field/openpmd_{step:0>6}.h5', 'r')
        E = data_step[f'data/{step}/fields/E/{efield_component}'][()].T
        data = np.append(data, E[x_index,y_index,z_index])
         
    x = np.linspace(-1.0,1.0,np.shape(E)[0])
    y = np.linspace(-1.0,1.0,np.shape(E)[1])
    z = np.linspace(-0.5,0.5,np.shape(E)[2])
    
    x_val = x[x_index]
    y_val = y[y_index]
    z_val = z[z_index]
    
    x = steps*dt
    y = data
    plt.figure(figsize=(10,10))
    plt.subplot(2, 1, 1)
    plt.plot(x*1e6,y,'k')
    plt.xlabel('t (us)')
    plt.ylabel(f'E$_{efield_component}$ for x={x_val:.2}, y={y_val:.2}, z={z_val:.2} m')
    plt.title(f'E$_{efield_component}$ Timeseries')
    plt.grid()

    N = len(x)
    T = np.mean(np.diff(x))
    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]

    plt.subplot(2, 1, 2)
    plt.plot(xf[1:]/(1.0e6), 2.0/N * np.abs(yf[1:N//2]),'k')
    plt.xlabel('Freq (MHz)')
    plt.ylabel('FFT Amplitude')
    plt.title(f'E$_{efield_component}$ FFT')
    plt.grid()
    plt.xticks(np.arange(min(xf/(1.0e6)), max(xf/(1.0e6))+1, 1.0))

    plt.tight_layout()
    plt.savefig(f'{figure_folder}/e{efield_component}_timeseries_{suffix}.png')
    plt.show()

In [ ]:
steps = np.arange(0,current_step+500,500)
efield_fft(filenames, figure_folder, steps, 'z', dt, 0, 0, 2, 'z1')

In [ ]:
def get_hist_max(filename, steps, species_name):
    max_xy = 0
    max_xz = 0
    directory_path = f'{filenames[0]}'
    for step in steps:
        data_step = h5.File(f'{directory_path}/particle/openpmd_{step:0>6}.h5', 'r')
        n_particles = len(data_step[f'data/{step}/particles/{species_name}/position/x'])
        if n_particles > 0:
            x = data_step[f'data/{step}/particles/{species_name}/position/x'][()]
            y = data_step[f'data/{step}/particles/{species_name}/position/y'][()]
            z = data_step[f'data/{step}/particles/{species_name}/position/z'][()]
            weighting = data_step[f'data/{step}/particles/{species_name}/weighting'][()]

            hist_xy, xedges, yedges = np.histogram2d(x*1e2, y*1e2, bins=(200, 200), 
                                                     range =[[-100, 100], [-100, 100]],
                                                     weights = weighting)
            max_xy = np.max(np.array([max_xy,np.max(hist_xy)]))

            hist_xz, xedges2, zedges = np.histogram2d(x*1e2, z*1e2, bins=(200, 100), 
                                                      range =[[-100, 100], [-50, 50]],
                                                      weights = weighting)
            max_xz = np.max(np.array([max_xz,np.max(hist_xz)]))
    print('max xy: ',max_xy)
    print('max xz: ',max_xz)
    
    return max_xy, max_xz

In [ ]:
def plot_histogram_2d(filename, figure_folder, dt, steps, species_name, rmax, max_xy, max_xz, extension):
    bin_num = 90
    directory_path = f'{filenames[0]}'
    for step in steps:
            
        data_step = h5.File(f'{directory_path}/particle/openpmd_{step:0>6}.h5', 'r')
        n_particles = len(data_step[f'data/{step}/particles/{species_name}/position/x'])
        if n_particles > 0:
            x = data_step[f'data/{step}/particles/{species_name}/position/x'][()]
            y = data_step[f'data/{step}/particles/{species_name}/position/y'][()]
            z = data_step[f'data/{step}/particles/{species_name}/position/z'][()]
            weighting = data_step[f'data/{step}/particles/{species_name}/weighting'][()]
            
            hist_xy, xedges, yedges = np.histogram2d(x*1e2, y*1e2, bins=(200, 200), 
                                                     range =[[-100, 100], [-100, 100]],
                                                     weights = weighting)
            # max_xy = np.max(hist_xy)
            hist_xy = hist_xy.T #/ max_xy
            
            hist_xz, xedges2, zedges = np.histogram2d(x*1e2, z*1e2, bins=(200, 100), 
                                                      range =[[-100, 100], [-50, 50]],
                                                      weights = weighting)
            # max_xz = np.max(hist_xz)
            hist_xz = hist_xz.T #/ max_xz

            fig = plt.figure(figsize=(20,5)) #14,6
            plt.subplot(1, 3, 1)
            plt.pcolormesh(xedges, yedges, hist_xy, cmap='magma', norm=mcolors.LogNorm(vmin=1.0e1, vmax=max_xy))#, vmin=0.0, vmax=max_xy)
            plt.colorbar(label='density [1/cm$^3$]',format='%.1e')
            plt.xlim(-100,100)
            plt.ylim(-100,100)
            plt.xlabel('x axis [cm]')
            plt.ylabel('y axis [cm]')
            # plt.title(f'Max density = {max_xy:.1e} 1/cm$^3$')            

            plt.subplot(1, 3, 2)
            plt.pcolormesh(xedges2, zedges, hist_xz, cmap='magma', norm=mcolors.LogNorm(vmin=1.0e1, vmax=max_xz))#, vmin=0.0, vmax=max_xz)
            plt.colorbar(label='density [1/cm$^3$]',format='%.1e')
            plt.xlim(-100,100)
            plt.ylim(-50,50)
            plt.xlabel('x axis [cm]')
            plt.ylabel('z axis [cm]')
            # plt.title(f'Max density = {max_xz:.1e} 1/cm$^3$')

            plt.subplot(1, 3, 3)
            x *= 1e2
            y *= 1e2
            z *= 1e2
            handles_list=[]
            labels_list=[]
            
            valid_indices = {'0' : np.where(np.logical_and(z <= 0.5, z >= -0.5))[0]}
            lower_bound = 0.5
            upper_bound = 1.5
            for key in np.arange(1,41,2):
                valid_indices[str(key)] = np.where(np.logical_and(z > lower_bound, z <= upper_bound))[0]
                valid_indices[str(key+1)] = np.where(np.logical_and(z < -lower_bound, z >= -upper_bound))[0]
                lower_bound += 1
                upper_bound += 1
                    
            normalized_hists = {}
            bin_centerss = {}
                    
            index = 0
            for key in valid_indices.keys():
                if len(valid_indices[key]) > 0:
                    r = np.sqrt(x[valid_indices[key]]**2.0 + y[valid_indices[key]]**2.0)
                    hist, bin_edges = np.histogram(r, bin_num, (0.0, rmax), weights = weighting[valid_indices[key]])
                    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
                    normalized_hist = hist / (2.0*np.pi*bin_centers*np.diff(bin_edges))
                            
                    bin_centerss[str(index)] = bin_centers
                    normalized_hists[str(index)] = normalized_hist
                else:
                    bin_centerss[str(index)] = np.array([0])
                    normalized_hists[str(index)] = np.array([0])
                index += 1
                    
            # plot here   
            index = 0
            divide = 0
            normalized_hist = 0
            for key in valid_indices.keys():
                if index == 0:
                    hydrogen1_label, = plt.plot(bin_centerss[str(index)], normalized_hists[str(index)])#, color=color)
                    handles_list.append(hydrogen1_label)
                    labels_list.append('D, |z|$\leq$0.5')
                else:
                    if len(valid_indices[key]) > 0:
                        normalized_hist += normalized_hists[str(index)]
                        bin_centers = bin_centerss[str(index)]
                        divide += 1
                    if index % 2 == 0 and divide > 0:
                        mean_hist = normalized_hist / divide
                        hydrogen1_mean, = plt.plot(bin_centers, mean_hist)#, '--', color=color)
                        handles_list.append(hydrogen1_mean)
                        labels_list.append(f'D, {0.5+int((index-1)/2)}<|z|$\leq${1.5+int((index-1)/2)}')
                                
                        normalized_hist = 0
                        divide = 0
                index += 1
            
            plt.yscale('log')
            plt.ylim(1.0e4,1.0e10)
            plt.xlim(0.0,rmax)
            plt.xlabel('r=$\sqrt{x^2 + y^2}$ (cm)')
            plt.ylabel('number density (cm$^{-3}$)')
            plt.tick_params(axis="both", which="both", direction="in")
            plt.grid() 
            
            try:
                plt.legend(handles=handles_list, labels=labels_list,
                           # loc='lower center', bbox_to_anchor=(0.5, -0.5), 
                           loc='center left', bbox_to_anchor=(1, 0.5),
                           ncol=2,prop={'size': 9})
            except:
                print('no label yet')   
         
            
            fig.text(0.03, 0.95, f'num macroparticles:     {len(weighting):.2e}') 
            fig.text(0.03, 0.91, f'num physical particles: {np.sum(weighting):.2e}') 
            plt.suptitle(f'{species_name} | t = {dt*step*1e6:.2f} $\mu$s')
            plt.tight_layout()
            
            if (1e6*dt*step) < 10.0:
                plt.savefig(f'{figure_folder}/hist_{species_name}/histograms_2d_0{1e6*dt*step:.2f}us.png')
            else:
                plt.savefig(f'{figure_folder}/hist_{species_name}/histograms_2d_{1e6*dt*step:.2f}us.png')
            # plt.show()
            plt.close(fig)
            
        else:
            print(f'No {species_name} particles at step {step}')

In [ ]:
step_min = 1200000
print(step_min*dt*1e6)

In [ ]:
steps = np.arange(step_min,current_step+500,500)

In [ ]:
species_name = 'deuterium'
max_xy, max_xz = get_hist_max(filenames[0], steps, species_name)

rmax = 80
extension = 'lognorm'
plot_histogram_2d(filenames[0], figure_folder, dt, steps, species_name, rmax, max_xy, max_xz, extension)

In [ ]:
species_name = 'argon'
max_xy, max_xz = get_hist_max(filenames[0], steps, species_name)

plot_histogram_2d(filenames[0], figure_folder, dt, steps, species_name, rmax, max_xy, max_xz, extension)

In [ ]:
species_name = 'electron'
max_xy, max_xz = get_hist_max(filenames[0], steps, species_name)

plot_histogram_2d(filenames[0], figure_folder, dt, steps, species_name, rmax, max_xy, max_xz, extension)

In [ ]:
import os
# os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
import moviepy.video.io.ImageSequenceClip

In [ ]:
def make_video(image_folder,fps,video_name):
    image_files = [os.path.join(image_folder,img)
                   for img in sorted(os.listdir(image_folder))
                   if img.startswith("histE_6")] + [os.path.join(image_folder,img)
                   for img in sorted(os.listdir(image_folder))
                   if img.startswith("histE_7")] + [os.path.join(image_folder,img)
                   for img in sorted(os.listdir(image_folder))
                   if img.startswith("histE_8")] + [os.path.join(image_folder,img)
                   for img in sorted(os.listdir(image_folder))
                   if img.startswith("histE_9")] + [os.path.join(image_folder,img)
                   for img in sorted(os.listdir(image_folder))
                   if img.startswith("histE_1")]
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
    clip.write_videofile(f'{video_name}.mp4')

In [ ]:
fps = 15
for filename in filenames:
    for diag in np.array(['hist_deuterium', 
                          'hist_argon', 
                          'hist_electron']):
        image_folder = f'{figure_folder}/{diag}'
        video_name = f'{figure_folder}/{diag}'
        make_video(image_folder,fps,video_name)

In [ ]:
step_min = 1200000 #current_step # 
steps = np.arange(step_min,current_step+500,500)

# Make energy histograms
joule_to_eV = 6.242e+18
mass_deuterium = 3.3435837768e-27 #kg
mass_e = 9.11e-31

sn = 'deuterium'
mass = mass_deuterium
directory_path = f'{filenames[0]}'

Emax = 0.0
for i,step in enumerate(steps):
    E_master = []
    for j,filename in enumerate(filenames):
        data_step = h5.File(f'{directory_path}/particle/openpmd_{step:0>6}.h5', 'r')
        num_part = len(data_step[f'data/{step}/particles/{sn}/momentum/x'])
        if num_part > 0:
            vx = data_step[f'data/{step}/particles/{sn}/momentum/x'][()] / mass
            vy = data_step[f'data/{step}/particles/{sn}/momentum/y'][()] / mass
            vz = data_step[f'data/{step}/particles/{sn}/momentum/z'][()] / mass
            E_master.append(0.5 * mass * (vx**2+vy**2+vz**2) * joule_to_eV / 1e3) #keV
        else:
            E_master.append(np.array([0.0]))
    Emax = np.max(np.array([np.max(E_master),Emax]))

for i,step in enumerate(steps):
    
    time = steps[i]*dt*1e6
    E_master = []
    fig = plt.figure(figsize=(10,5))
    plt.subplot(1, 1, 1)
    plt.xlabel('E [keV]')
    plt.ylabel('macroparticle PDF')
    plt.title(f't = {time:.3f} us')
    for j,filename in enumerate(filenames):
        data_step = h5.File(f'{directory_path}/particle/openpmd_{step:0>6}.h5', 'r')
        num_part = len(data_step[f'data/{step}/particles/{sn}/momentum/x'])
        if num_part > 0:
            vx = data_step[f'data/{step}/particles/{sn}/momentum/x'][()] / mass
            vy = data_step[f'data/{step}/particles/{sn}/momentum/y'][()] / mass
            vz = data_step[f'data/{step}/particles/{sn}/momentum/z'][()] / mass
            weights = data_step[f'data/{step}/particles/{sn}/weighting'][()]
            E_master.append(0.5 * mass * (vx**2+vy**2+vz**2) * joule_to_eV / 1e3) #keV
        else:
            E_master.append(np.array([0.0]))

    counts = []
    for j,_ in enumerate(filenames):
        counts.append([])
        
        # range=(0.0,60.0), 
        frq, edges = np.histogram(E_master[j], bins=1000, range=(0.0,Emax), density=True) #weights=weights, range=(28.5,32.0), 
        if j == 2:
            plt.bar(edges[:-1], frq/np.max(frq), width=np.diff(edges), align="edge")#, label=labels[j])
        else:
            plt.bar(edges[:-1], frq/np.max(frq), width=np.diff(edges), alpha=0.75, align="edge")#, label=labels[j])

    plt.xlim(0.0,Emax)
    plt.ylim(0,1)
    plt.grid()
    fig.text(0.09, 0.81, f'total species KE [keV]: {np.sum(E_master):.2e}')
    fig.text(0.09, 0.86, f'mean macro KE [keV]: {np.sum(E_master)/num_part:.2f}') 
    plt.tight_layout()
    # plt.show()
    if (1e6*dt*step) < 10.0:
        plt.savefig(f'{figure_folder}/histE_dp/histE_0{1e6*dt*step:.5f}us.png')
    else:
        plt.savefig(f'{figure_folder}/histE_dp/histE_{1e6*dt*step:.5f}us.png')
    plt.close(fig) 

In [ ]:
step_min = 1200000 #current_step # 
steps = np.arange(step_min,current_step+500,500)

# Make energy histograms
joule_to_eV = 6.242e+18
mass_deuterium = 3.3435837768e-27 #kg
mass_e = 9.11e-31

sn = 'electron'
mass = mass_e
directory_path = f'{filenames[0]}'

Emax = 0.0
for i,step in enumerate(steps):
    E_master = []
    for j,filename in enumerate(filenames):
        data_step = h5.File(f'{directory_path}/particle/openpmd_{step:0>6}.h5', 'r')
        num_part = len(data_step[f'data/{step}/particles/{sn}/momentum/x'])
        if num_part > 0:
            vx = data_step[f'data/{step}/particles/{sn}/momentum/x'][()] / mass
            vy = data_step[f'data/{step}/particles/{sn}/momentum/y'][()] / mass
            vz = data_step[f'data/{step}/particles/{sn}/momentum/z'][()] / mass
            E_master.append(0.5 * mass * (vx**2+vy**2+vz**2) * joule_to_eV)# / 1e3) #keV
        else:
            E_master.append(np.array([0.0]))
    Emax = np.max(np.array([np.max(E_master),Emax]))
    
for i,step in enumerate(steps):
    
    time = steps[i]*dt*1e6
    E_master = []
    fig = plt.figure(figsize=(10,5))
    plt.subplot(1, 1, 1)
    plt.xlabel('E [eV]')
    plt.ylabel('macroparticle PDF')
    plt.title(f't = {time:.3f} us')
    for j,filename in enumerate(filenames):
        data_step = h5.File(f'{directory_path}/particle/openpmd_{step:0>6}.h5', 'r')
        num_part = len(data_step[f'data/{step}/particles/{sn}/momentum/x'])
        if num_part > 0:
            vx = data_step[f'data/{step}/particles/{sn}/momentum/x'][()] / mass
            vy = data_step[f'data/{step}/particles/{sn}/momentum/y'][()] / mass
            vz = data_step[f'data/{step}/particles/{sn}/momentum/z'][()] / mass
            weights = data_step[f'data/{step}/particles/{sn}/weighting'][()]
            E_master.append(0.5 * mass * (vx**2+vy**2+vz**2) * joule_to_eV )#/ 1e3) #keV
        else:
            E_master.append(np.array([0.0]))

    counts = []
    for j,_ in enumerate(filenames):
        counts.append([])
        
        # range=(0.0,150.0), 
        frq, edges = np.histogram(E_master[j], bins=1000, range=(0.0,Emax), density=True) #weights=weights, range=(28.5,32.0), 
        if j == 2:
            plt.bar(edges[:-1], frq/np.max(frq), width=np.diff(edges), align="edge")#, label=labels[j])
        else:
            plt.bar(edges[:-1], frq/np.max(frq), width=np.diff(edges), alpha=0.75, align="edge")#, label=labels[j])

    plt.xlim(-10.0,Emax)
    plt.ylim(0,1)
    plt.grid()
    fig.text(0.09, 0.81, f'total species KE [eV]: {np.sum(E_master):.2e}')
    fig.text(0.09, 0.86, f'mean macro KE [eV]: {np.sum(E_master)/num_part:.2f}') 
    plt.tight_layout()
    # plt.show()
    if (1e6*dt*step) < 10.0:
        plt.savefig(f'{figure_folder}/histE_e/histE_0{1e6*dt*step:.5f}us.png')
    else:
        plt.savefig(f'{figure_folder}/histE_e/histE_{1e6*dt*step:.5f}us.png')
    plt.close(fig) 

In [ ]:
fps = 15
for filename in filenames:
    for diag in np.array(['histE_dp']): #,'histE_e']):
        #'histograms_deuterium_2d', 'histograms_ar_ions_2d', 'histograms_electrons_2d'
        image_folder = f'{figure_folder}/{diag}'
        video_name = f'{figure_folder}/{diag}'
        make_video(image_folder,fps,video_name)

In [ ]:
# def get_Ehist_max(filename, steps):
#     max_Ez0z = 0
#     min_Ez0z = 0
#     max_Ez0y = 0
#     min_Ez0y = 0
#     max_Ex0z = 0
#     min_Ex0z = 0
    
#     x = np.linspace(-1.0,1.0,64)
#     y = np.linspace(-1.0,1.0,64)
#     z = np.linspace(-0.5,0.5,32)
    
#     directory_path = f'{filenames[0]}'
#     for step in steps:
#         data_step = h5.File(f'{directory_path}/field/openpmd_{step:0>6}.h5', 'r')
#         Ex = data_step[f'data/{step}/fields/E/x'][()].T
#         # Ey = data_step[f'data/{step}/fields/E/y'][()].T
#         Ez = data_step[f'data/{step}/fields/E/z'][()].T
            
#         max_Ez0z = np.max(np.array([max_Ez0z,np.max(Ez[:,:,int(len(z)/2)])]))
#         min_Ez0z = np.min(np.array([max_Ez0z,np.min(Ez[:,:,int(len(z)/2)])]))

#         max_Ez0y = np.max(np.array([max_Ez0y,np.max(Ez[:,int(len(y)/2),:])]))
#         min_Ez0y = np.min(np.array([max_Ez0y,np.min(Ez[:,int(len(y)/2),:])]))
        
#         max_Ex0z = np.max(np.array([max_Ex0z,np.max(Ex[:,:,int(len(z)/2)])]))
#         min_Ex0z = np.min(np.array([max_Ex0z,np.min(Ex[:,:,int(len(z)/2)])]))
        
#     # print('max xy: ',max_xy)
#     # print('max xz: ',max_xz)
    
#     return max_Ez0z, min_Ez0z, max_Ez0y, min_Ez0y, max_Ex0z, min_Ex0z

In [ ]:
# def elec_field(filename, figure_folder, dt, steps,
#                max_Ez0z, min_Ez0z, max_Ez0y, min_Ez0y, max_Ex0z, min_Ex0z):
#     x = np.linspace(-1.0,1.0,64)
#     y = np.linspace(-1.0,1.0,64)
#     z = np.linspace(-0.5,0.5,32)
    
#     directory_path = f'{filename}'
#     for step in steps:
    
#         data_step = h5.File(f'{directory_path}/field/openpmd_{step:0>6}.h5', 'r')
#         Ex = data_step[f'data/{step}/fields/E/x'][()].T
#         # Ey = data_step[f'data/{step}/fields/E/y'][()].T
#         Ez = data_step[f'data/{step}/fields/E/z'][()].T

#         fig = plt.figure(figsize=(20,5))
#         plt.subplot(1, 3, 1)
#         plt.pcolormesh(x*1e2, y*1e2, Ez[:,:,int(len(z)/2)].T, vmin=min_Ez0z, vmax=max_Ez0z)
#         #, cmap='magma')#, norm=mcolors.LogNorm(vmin=1.0e1, vmax=max_xy))#, vmin=0.0, vmax=max_xy)
#         plt.colorbar(format='%.1e')
#         plt.xlim(-100,100)
#         plt.ylim(-100,100)
#         plt.xlabel('x axis [cm]')
#         plt.ylabel('y axis [cm]')
#         plt.title(f'Ez, z=0 | max: {np.max(Ez[:,:,int(len(z)/2)]):.2e} | min: {np.min(Ez[:,:,int(len(z)/2)]):.2e}')

#         plt.subplot(1, 3, 2)
#         plt.pcolormesh(x*1e2, z*1e2, Ez[:,int(len(y)/2),:].T, vmin=min_Ez0y, vmax=max_Ez0y)
#         #, cmap='magma')#, norm=mcolors.LogNorm(vmin=1.0e1, vmax=max_xy))#, vmin=0.0, vmax=max_xy)
#         plt.colorbar(format='%.1e')
#         plt.xlim(-100,100)
#         plt.ylim(-50,50)
#         plt.xlabel('x axis [cm]')
#         plt.ylabel('z axis [cm]')
#         plt.title(f'Ez, y=0 | max: {np.max(Ez[:,int(len(y)/2),:]):.2e} | min: {np.min(Ez[:,int(len(y)/2),:]):.2e}')

#         plt.subplot(1, 3, 3)
#         plt.pcolormesh(x*1e2, y*1e2, Ex[:,:,int(len(z)/2)].T, vmin=min_Ex0z, vmax=max_Ex0z)
#         #, cmap='magma')#, norm=mcolors.LogNorm(vmin=1.0e1, vmax=max_xy))#, vmin=0.0, vmax=max_xy)
#         plt.colorbar(format='%.1e')
#         plt.xlim(-100,100)
#         plt.ylim(-100,100)
#         plt.xlabel('x axis [cm]')
#         plt.ylabel('y axis [cm]')
#         plt.title(f'Ex, z=0 | max: {np.max(Ex[:,:,int(len(z)/2)]):.2e} | min: {np.min(Ex[:,:,int(len(z)/2)]):.2e}')

#         plt.suptitle(f'Electric field | t = {dt*step*1e6:.2f} $\mu$s') #at z = 0 (top row) and y = 0 (bottom row) 
#         plt.tight_layout()
#         # plt.show()
#         if (1e6*dt*step) < 10.0:
#             plt.savefig(f'{figure_folder}/Efield_3/Efield_0{1e6*dt*step:.5f}us.png')
#         else:
#             plt.savefig(f'{figure_folder}/Efield_3/Efield_3_{1e6*dt*step:.5f}us.png')
#         plt.close(fig) 

In [ ]:
# step_min = 800000 #current_step # 
# steps = np.arange(step_min,current_step+500,500)

# max_Ez0z, min_Ez0z, max_Ez0y, min_Ez0y, max_Ex0z, min_Ex0z = get_Ehist_max(filename, steps)

In [ ]:
# step_min = 800000 #current_step # 
# steps = np.arange(step_min,current_step+500,500)

# elec_field(filenames[0], figure_folder, dt, steps,
#            max_Ez0z, min_Ez0z, max_Ez0y, min_Ez0y, max_Ex0z, min_Ex0z)

In [ ]:
# fps = 15
# for filename in filenames:
#     for diag in np.array(['Efield_3']):
#         image_folder = f'{figure_folder}/{diag}'
#         video_name = f'{figure_folder}/{diag}'
#         make_video(image_folder,fps,video_name)